In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/movies_metadata.csv')
df.columns

/tmp/ipykernel_172709/1819800651.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/movies_metadata.csv')


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [3]:
# get the relevant columns only
df = df[['title', 'genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

In [4]:
df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [5]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')


In [6]:
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [7]:
def convert_int(x):
    try:
        return int(x)
    except:
        return 0
    
df['year'] = df['year'].apply(convert_int)

In [8]:
df = df.drop('release_date', axis = 1)
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


In [11]:
type(df.iloc[0]['genres']), df.iloc[0]['genres']

(str,
 "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]")

In [12]:
from ast import literal_eval
a = "[1, 2, 3]"
print(type(a))

b = literal_eval(a)
print(type(b))


<class 'str'>
<class 'list'>


In [13]:
# fill all NaN values with empty list
df['genres'] = df['genres'].fillna('[]')

In [14]:
# apply literal_eval to convert to the list object
df['genres'] = df['genres'].apply(literal_eval)

In [15]:
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[Animation, Comedy, Family]",81.0,7.7,5415.0,1995
1,Jumanji,"[Adventure, Fantasy, Family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[Romance, Comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[Comedy, Drama, Romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[Comedy],106.0,5.7,173.0,1995


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45460 non-null  object 
 1   genres        45466 non-null  object 
 2   runtime       45203 non-null  float64
 3   vote_average  45460 non-null  float64
 4   vote_count    45460 non-null  float64
 5   year          45466 non-null  int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 2.1+ MB


In [17]:
s = df.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'

In [19]:
gen_df = df.drop('genres', axis = 1)
gen_df.head()

,title,runtime,vote_average,vote_count,year
0,Toy Story,81.0,7.7,5415.0,1995
1,Jumanji,104.0,6.9,2413.0,1995
2,Grumpier Old Men,101.0,6.5,92.0,1995
3,Waiting to Exhale,127.0,6.1,34.0,1995
4,Father of the Bride Part II,106.0,5.7,173.0,1995


In [22]:
gen_df = gen_df.join(s)
gen_df.head()

,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,Animation
0,Toy Story,81.0,7.7,5415.0,1995,Comedy
0,Toy Story,81.0,7.7,5415.0,1995,Family
1,Jumanji,104.0,6.9,2413.0,1995,Adventure
1,Jumanji,104.0,6.9,2413.0,1995,Fantasy


In [53]:
gen_df.genre = gen_df.genre.astype(str)
gen_df.genre = gen_df.genre.apply(lambda x: x.lower())
gen_df.head()


,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,animation
0,Toy Story,81.0,7.7,5415.0,1995,comedy
0,Toy Story,81.0,7.7,5415.0,1995,family
1,Jumanji,104.0,6.9,2413.0,1995,adventure
1,Jumanji,104.0,6.9,2413.0,1995,fantasy


## Build Chart Function

- Get user input on their preferences
- Extract all the movies that match the conditions set by the user
- Calculate the values of m and C for only those movies and proceed to build the chart as in the previous section

In [57]:
def build_chart(gen_df, percentile=0.8):
    # get the relevant input preferences
    genre = input("Enter preffered genre: ").lower()
    low_time = int(input("Enter shortest duration: "))
    high_time = int(input("Enter longest duration: "))
    low_year = int(input("Enter earliest year: "))
    high_year = int(input("Enter latest year: "))

    # filter out the movies that satisfy the preferences
    movies = gen_df.copy()
    movies = movies[(movies['genre'] == genre) &
                    (movies['runtime'] >= low_time) &
                    (movies['runtime'] <= high_time) &
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]

    print(movies.info())
    
    # compute values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    q_movies = movies.copy().loc[movies['vote_count'] >= m]

    def weighted_rating(x, m=m, C=C):
        v = x['vote_count']
        R = x['vote_average']
        return (v / (v + m) * R) + (m / (v + m) * C)


    q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

    # sort the movies on the basis of score
    q_movies = q_movies.sort_values('score', ascending=False)
    return q_movies

In [58]:
result = build_chart(gen_df)
result.head()

<class 'pandas.core.frame.DataFrame'>
Index: 439 entries, 0 to 45449
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         439 non-null    object 
 1   runtime       439 non-null    float64
 2   vote_average  439 non-null    float64
 3   vote_count    439 non-null    float64
 4   year          439 non-null    int64  
 5   genre         439 non-null    object 
dtypes: float64(3), int64(1), object(2)
memory usage: 24.0+ KB
None


,title,runtime,vote_average,vote_count,year,genre,score
9698,Howl's Moving Castle,119.0,8.2,2049.0,2004,animation,7.994823
359,The Lion King,89.0,8.0,5520.0,1994,animation,7.926672
0,Toy Story,81.0,7.7,5415.0,1995,animation,7.637500
6232,Finding Nemo,100.0,7.6,6292.0,2003,animation,7.549423
546,The Nightmare Before Christmas,76.0,7.6,2135.0,1993,animation,7.460500
